In [1]:
from tqdm import tqdm
import pymssql
import pandas as pd
import numpy as np
import warnings
import glob 
import os

warnings.filterwarnings('ignore')

In [2]:
conn = pymssql.connect(server='AX-SQL', database='Staging')
cursor = conn.cursor()

sql = f'''
SELECT [Код склада], [Код товара], [Артикул поставщика], [Внутренний артикул], [ID Сайта], [Товарное направление],
[Товарная группа], [Товарный кластер], [Проба], [Размер изделия], [Тип изделия 1], [Тип изделия 2],
[Тип изделия 3], [Дизайн], [Бренд], [Гендерный признак], [Ценовая корзина], [Группа цен], [Остаток], [Цена Розн., за шт],
[Чистый вес], [Цена изделия на бирке], [Скидка на изделии], [Запрет скидки], [Цена Закупки за 1 шт. (упр сс с НДС за единицу)], [Дата закупки]
from [Staging].[Reports].[Remainings_8h]
where [Товарное направление] in (N'БК', N'ЦБ', N'ИФ', N'ПДК', N'ДК')
'''

sql_query = pd.read_sql_query(sql, conn)
stock = pd.DataFrame(sql_query)

In [3]:
images = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\Артикул-фото 2024.06.11_.xlsx')

In [4]:
zapret = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\запрет Артикулы к размещению.xlsx')

In [5]:
kniga = pd.read_excel(r'C:\Users\Trenkin.Sergey\Desktop\Книга11.xlsx')

In [6]:
os.chdir(r'C:\\Users\Trenkin.Sergey\Desktop\sales')
extension = 'xlsx'
all_filenames = [i for i in glob.glob('*.{}'.format(extension))]
all_filenames

['2024-03-10_2024-03-19.xlsx',
 '2024-03-20_2024-03-31.xlsx',
 '2024-04-01_2024-04-09.xlsx',
 '2024-04-10_2024-04-19.xlsx',
 '2024-04-20_2024-04-30.xlsx',
 '2024-05-01_2024-05-09.xlsx',
 '2024-05-10_2024-05-19.xlsx',
 '2024-05-20_2024-05-31.xlsx',
 '2024-06-01_2024-06-09.xlsx',
 '2024-06-10_2024-06-10.xlsx',
 '2024-06-10_2024-06-12.xlsx',
 '2024-06-10_2024-06-18.xlsx']

In [7]:
sale = pd.DataFrame()
for name in tqdm(all_filenames):
    sales = pd.read_excel(name)
    sale = pd.concat([sales, sale], ignore_index= True)

100%|██████████| 12/12 [11:51<00:00, 59.27s/it]


In [8]:
sale['Дата'] = pd.to_datetime(sale['Дата'], dayfirst=True, format='mixed')

In [9]:
sale = sale[(sale['Дата'] >= pd.to_datetime('2024-03-14', dayfirst=True, format='mixed')) & 
            (sale['Дата'] <= pd.to_datetime('2024-06-19', dayfirst=True, format='mixed'))]

In [10]:
sale = sale.groupby(by = 'Тов', as_index=False)[['Количество', 'Общая сумма']].sum()

In [11]:
ids = stock[['Код товара', 'ID Сайта']]
ids.columns = ['Тов', 'ID Сайта']

In [12]:
sale = sale.merge(ids, how='left', on='Тов')
sale = sale.drop(columns=['Тов'])
sale = sale[['ID Сайта', 'Количество', 'Общая сумма']]

In [13]:
kniga['Код склада'] = kniga['Код склада'].astype(str)
skladi = kniga['Код склада'].unique().tolist()

In [14]:
stock.columns

Index(['Код склада', 'Код товара', 'Артикул поставщика', 'Внутренний артикул',
       'ID Сайта', 'Товарное направление', 'Товарная группа',
       'Товарный кластер', 'Проба', 'Размер изделия', 'Тип изделия 1',
       'Тип изделия 2', 'Тип изделия 3', 'Дизайн', 'Бренд',
       'Гендерный признак', 'Ценовая корзина', 'Группа цен', 'Остаток',
       'Цена Розн., за шт', 'Чистый вес', 'Цена изделия на бирке',
       'Скидка на изделии', 'Запрет скидки',
       'Цена Закупки за 1 шт. (упр сс с НДС за единицу)', 'Дата закупки'],
      dtype='object')

In [15]:
stock.columns = ['code', 'Код товара', 'Артикул поставщика', 'Внутренний артикул',
    'ID Сайта', 'Товарное направление', 'Товарная группа',
    'Товарный кластер', 'Проба', 'Размер изделия', 'Тип изделия 1',
    'Тип изделия 2', 'Тип изделия 3', 'Дизайн', 'Бренд',
    'Гендерный признак', 'Ценовая корзина', 'Группа цен', 'Остаток', 'Цена Розн., за шт',
    'Чистый вес', 'Цена изделия на бирке', 'Скидка на изделии',
    'Запрет скидки', 'Цена Закупки за 1 шт. (упр сс с НДС за единицу)',
    'Дата закупки']

In [16]:
stock = stock.query('code in @skladi')

In [17]:
stock.columns = ['Код склада', 'Код товара', 'Артикул поставщика', 'Внутренний артикул',
    'ID Сайта', 'Товарное направление', 'Товарная группа',
    'Товарный кластер', 'Проба', 'Размер изделия', 'Тип изделия 1',
    'Тип изделия 2', 'Тип изделия 3', 'Дизайн', 'Бренд',
    'Гендерный признак', 'Ценовая корзина', 'Группа цен', 'Остаток', 'Цена Розн., за шт',
    'Чистый вес', 'Цена изделия на бирке', 'Скидка на изделии',
    'Запрет скидки', 'Цена Закупки за 1 шт. (упр сс с НДС за единицу)',
    'Дата закупки']

In [18]:
result = stock.drop_duplicates(subset=['ID Сайта'])

In [19]:
tovs = result['ID Сайта'].dropna().tolist()

In [20]:
images['article'] = images['article'].astype(str)

In [21]:
images = images.query('article in @tovs')

In [22]:
images = images[['article', 'published', 'Ссылка на изображение']]

In [23]:
images.columns = ['ID Сайта', 'Опубликован', 'Ссылка на изображение']

In [24]:
result = result.merge(images, how='outer', on='ID Сайта')

In [25]:
result = result.drop(columns = ['Остаток'])

In [26]:
df = stock

In [27]:
stock = df.groupby(by = 'ID Сайта', as_index=False)['Остаток'].sum()

In [28]:
result = result.merge(stock, how='left', on='ID Сайта')

In [29]:
sale.columns = ['ID Сайта', 'Количество продаж 14.03-19.06', 'Сумма продаж 14.03-19.06']

In [30]:
result = result.merge(sale, how='left', on='ID Сайта')

In [31]:
result.columns

Index(['Код склада', 'Код товара', 'Артикул поставщика', 'Внутренний артикул',
       'ID Сайта', 'Товарное направление', 'Товарная группа',
       'Товарный кластер', 'Проба', 'Размер изделия', 'Тип изделия 1',
       'Тип изделия 2', 'Тип изделия 3', 'Дизайн', 'Бренд',
       'Гендерный признак', 'Ценовая корзина', 'Группа цен',
       'Цена Розн., за шт', 'Чистый вес', 'Цена изделия на бирке',
       'Скидка на изделии', 'Запрет скидки',
       'Цена Закупки за 1 шт. (упр сс с НДС за единицу)', 'Дата закупки',
       'Опубликован', 'Ссылка на изображение', 'Остаток',
       'Количество продаж 14.03-19.06', 'Сумма продаж 14.03-19.06'],
      dtype='object')

In [32]:
result = result.drop_duplicates(subset=['ID Сайта'])

In [33]:
df['Чистый вес'] = df['Чистый вес'].astype(float)

In [34]:
clear_weight = df.groupby(by = 'ID Сайта', as_index=False)['Чистый вес'].agg({'min', 'median', 'max'})

In [35]:
clear_weight

,ID Сайта,max,median,min
0,,21.280,1.974,0.206
1,1000054,1.150,1.080,0.980
2,1000091,1.154,1.059,0.944
3,1000339,2.252,2.162,2.092
4,1000450,3.540,3.540,3.540
...,...,...,...,...
44603,8380055,2.140,2.090,2.050
44604,8380061,1.708,1.700,1.670
44605,8380063,1.930,1.770,1.510
44606,8380071,2.610,2.520,2.370


In [43]:
clear_weight = df.groupby(by = 'ID Сайта', as_index=False)['Чистый вес'].agg({'min', 'median', 'max'})
clear_weight.columns = ['ID Сайта', 'Чистый вес макс', 'Чистый вес медиана', 'Чистый вес мин']
clear_weight['Чистый вес размах, %'] = (clear_weight['Чистый вес макс'] / clear_weight['Чистый вес мин'] - 1) 

In [44]:
clear_weight

,ID Сайта,Чистый вес макс,Чистый вес медиана,Чистый вес мин,"Чистый вес размах, %"
0,,21.280,1.974,0.206,102.300971
1,1000054,1.150,1.080,0.980,0.173469
2,1000091,1.154,1.059,0.944,0.222458
3,1000339,2.252,2.162,2.092,0.076482
4,1000450,3.540,3.540,3.540,0.000000
...,...,...,...,...,...
44603,8380055,2.140,2.090,2.050,0.043902
44604,8380061,1.708,1.700,1.670,0.022754
44605,8380063,1.930,1.770,1.510,0.278146
44606,8380071,2.610,2.520,2.370,0.101266


In [45]:
result = result.merge(clear_weight, how='left', on='ID Сайта')

In [54]:
result = result[result['ID Сайта'] != '']
df = df[df['ID Сайта'] != '']

In [56]:
price_birka = df.groupby(by = 'ID Сайта', as_index=False)['Цена Розн., за шт'].agg({'min', 'median', 'max'})
price_birka.columns = ['ID Сайта', 'Цена изделия на бирке макс', 'Цена изделия на бирке медиана', 'Цена изделия на бирке мин']

In [57]:
price_birka

,ID Сайта,Цена изделия на бирке макс,Цена изделия на бирке медиана,Цена изделия на бирке мин
0,1000054,30225.0,20249.0,19774.0
1,1000091,49205.0,37888.0,6442.0
2,1000339,118218.0,115330.0,104042.0
3,1000450,294990.0,294990.0,294990.0
4,1000477,19900.0,19900.0,19900.0
...,...,...,...,...
44602,8380055,48832.0,47256.0,40680.0
44603,8380061,40623.0,37566.0,31895.0
44604,8380063,71439.0,38843.0,30317.0
44605,8380071,61569.0,48128.0,42957.0


In [58]:
result = result.merge(price_birka, how='left', on='ID Сайта')

In [59]:
base_discount = df.groupby(by = 'ID Сайта', as_index=False)['Скидка на изделии'].agg({'min', 'max'})
base_discount.columns = ['ID Сайта', 'Базовая скидка макс', 'Базовая скидка мин']

In [60]:
base_discount

,ID Сайта,Базовая скидка макс,Базовая скидка мин
0,1000054,57.0,53.0
1,1000091,66.0,0.0
2,1000339,67.0,62.0
3,1000450,0.0,0.0
4,1000477,0.0,0.0
...,...,...,...
44602,8380055,45.0,37.0
44603,8380061,38.0,37.0
44604,8380063,66.0,32.0
44605,8380071,59.0,24.0


In [61]:
result = result.merge(base_discount, how='left', on='ID Сайта')

In [62]:
df['Цена со скидками'] = np.where(
    df['Запрет скидки'] == 'Да', df['Цена Розн., за шт'], df['Цена Розн., за шт'] * (1-df['Скидка на изделии']/100) * 0.8 * 0.6
)

In [64]:
price_skidki = df.groupby(by = 'ID Сайта', as_index=False)['Цена со скидками'].agg({'min', 'median', 'max'})
price_skidki.columns = ['ID Сайта', 'Цена со скидками макс', 'Цена со скидками медиана', 'Цена со скидками мин']
price_skidki['Цена со скидками размах, %'] = (price_skidki['Цена со скидками макс'] / price_skidki['Цена со скидками мин'] - 1) 

In [66]:
result = result.merge(price_skidki, how='left', on='ID Сайта')

In [67]:
selfprice = df.groupby(by = 'ID Сайта', as_index=False)['Цена Закупки за 1 шт. (упр сс с НДС за единицу)'].agg({'min', 'median', 'max'})
selfprice.columns = ['ID Сайта', 'Себестоимость макс', 'Себестоимость медиана', 'Себестоимость мин']
selfprice['Себестоимость размах, %'] = (selfprice['Себестоимость макс'] / selfprice['Себестоимость мин'] - 1) 

In [68]:
selfprice

,ID Сайта,Себестоимость макс,Себестоимость медиана,Себестоимость мин,"Себестоимость размах, %"
0,1000054,2324.99,2249.94,2197.16,0.058180
1,1000091,4295.00,4294.99,3181.09,0.350166
2,1000339,8275.26,8073.05,7914.98,0.045519
3,1000450,218357.44,218357.44,218357.44,0.000000
4,1000477,10568.98,9276.36,7726.71,0.367850
...,...,...,...,...,...
44602,8380055,6510.98,6300.78,4531.52,0.436820
44603,8380061,5416.40,5008.69,4142.79,0.307428
44604,8380063,5864.25,5179.11,3402.11,0.723710
44605,8380071,7482.00,5337.14,4944.87,0.513083


In [69]:
result = result.merge(selfprice, how='left', on='ID Сайта')

In [70]:
df['Наценка'] = df['Цена со скидками'] / df['Цена Закупки за 1 шт. (упр сс с НДС за единицу)'] - 1

In [71]:
markup = df.groupby(by = 'ID Сайта', as_index=False)['Наценка'].agg({'min', 'median', 'max'})
markup.columns = ['ID Сайта', 'Наценка макс', 'Наценка медиана', 'Наценка мин']

In [73]:
result = result.merge(markup, how='left', on='ID Сайта')

In [74]:
zapret['Артикул сайта'] = zapret['Артикул сайта'].astype(str)

In [75]:
zapret = zapret['Артикул сайта'].tolist()

In [76]:
result['Запрет к размещению'] = np.nan
for i in result.index:
    if result['ID Сайта'][i] in zapret:
        result['Запрет к размещению'][i] = True
    else:
        result['Запрет к размещению'][i] = False

In [77]:
ecom = pd.read_excel(r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\МЮР\ecom_stock_raw\Остатки 2024.06.19.xlsx')

In [78]:
path = r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\discounts_zolotoy\\'
list_of_files = glob.glob(path + '\*.xlsx') 
latest_file = max(list_of_files, key=os.path.getctime)

discount_ecom = pd.read_excel(latest_file)

In [79]:
discount_ecom.columns = ['article', 'Опубликован', 'discount_value_promotions_1', 'discount_value_promotions_2', 'discount_value_promotions_3', 'discount_value_promotions_4']

In [80]:
discount_ecom = discount_ecom.drop(columns=['discount_value_promotions_2', 'discount_value_promotions_3', 'discount_value_promotions_4'])

In [81]:
ecom = ecom.merge(discount_ecom, how='left', on='article')

In [82]:
def discount_index(discount):
    try:
        return 1 - discount / 100
    except:
        return 1

In [83]:
ecom.discount_value_promotions_1 = ecom.discount_value_promotions_1.apply(discount_index)

In [84]:
ecom['Цена со скидками Ecom'] = np.where(
    (ecom['price'] == ecom['discount_price']) & (ecom['discount_value_promotions_1'] == 1), ecom['discount_price'] * ecom['discount_value_promotions_1'], 
    np.where(
        ecom['discount_value_promotions_1'] == 0.5, ecom['discount_price'] * ecom['discount_value_promotions_1'], ecom['discount_price'] * ecom['discount_value_promotions_1'] * 0.8
    )
)

In [85]:
ecom['article'] = ecom['article'].astype(str)
ecom = ecom[['article', 'size', 'Цена со скидками Ecom']]

In [86]:
ecom = ecom.query('article in @tovs')

In [87]:
ecom.columns = ['ID Сайта', 'size', 'Цена со скидками Ecom']

In [88]:
result = result[result['ID Сайта'] != '']

In [89]:
result['ID Сайта'] = result['ID Сайта'].astype(int).astype(str)
result['Размер изделия'] = result['Размер изделия'].astype(str)

In [90]:
ecom['size'] = ecom['size'].fillna(0.0)
ecom['size'] = ecom['size'].astype(str)

In [91]:
ecom['merge'] = ecom['ID Сайта'] + ecom['size']

In [92]:
result['merge'] = result['ID Сайта'] + result['Размер изделия']

In [93]:
ecom = ecom.drop(columns=['ID Сайта', 'size'])

In [94]:
result = result.merge(ecom, how='outer', on='merge')

In [95]:
result = result.drop(columns=['merge'])

In [96]:
result = result.dropna(subset=['ID Сайта'])

In [97]:
df['Сумма в себестоимости на остатках'] = df['Остаток'] * df['Цена Закупки за 1 шт. (упр сс с НДС за единицу)']

In [98]:
summa_ost = df.groupby(by ='ID Сайта', as_index=False)['Сумма в себестоимости на остатках'].sum()

In [99]:
result = result.merge(summa_ost, how='left', on='ID Сайта')

In [100]:
df['Цена за грамм'] = df['Цена со скидками'] / df['Чистый вес']

In [101]:
price_per_gramm = df.groupby(by = 'ID Сайта', as_index=False)['Цена за грамм'].agg({'min', 'median', 'max'})
price_per_gramm.columns = ['ID Сайта', 'Цена за грамм макс', 'Цена за грамм медиана', 'Цена за грамм мин']

In [102]:
price_per_gramm

,ID Сайта,Цена за грамм макс,Цена за грамм медиана,Цена за грамм мин
0,1000054,5424.730435,4661.402449,4130.568889
1,1000091,10154.753299,8212.036877,6260.447036
2,1000339,9800.604713,8449.709528,8315.155950
3,1000450,83330.508475,83330.508475,83330.508475
4,1000477,6933.797909,6236.289564,5835.777126
...,...,...,...,...
44602,8380055,6904.308293,6902.533206,5042.028169
44603,8380061,7192.268852,6682.328471,5683.803593
44604,8380063,7602.581333,7042.098462,4675.393684
44605,8380071,6982.385116,4950.308571,4216.922429


In [103]:
result['Себестоимость размах, %'].value_counts()

Себестоимость размах, %
0.000000    11811
inf             5
2.790509        4
0.054348        3
0.031250        3
            ...  
0.112722        1
0.506075        1
0.586435        1
0.182068        1
0.235301        1
Name: count, Length: 32761, dtype: int64

In [104]:
result = result.merge(price_per_gramm, how='left', on='ID Сайта')

In [105]:
result.to_csv('result.csv', index=False)

In [106]:
zapret_sk = df.groupby(by = 'ID Сайта', as_index=False)['Запрет скидки'].count()

In [107]:
yes = df[df['Запрет скидки'] == 'Да']
no = df[df['Запрет скидки'] == 'Нет']

In [108]:
yes = yes.groupby(by = 'ID Сайта', as_index=False)['Запрет скидки'].count()
no = no.groupby(by = 'ID Сайта', as_index=False)['Запрет скидки'].count()

In [109]:
yes.columns = ['ID Сайта', 'Количество с запрещенной скидкой']

In [110]:
no.columns = ['ID Сайта', 'Количество с разрешенной скидкой']

In [111]:
yes = yes.merge(no, how='outer', on='ID Сайта')

In [112]:
result = result.merge(yes, how='left', on='ID Сайта')

In [113]:
result = result.drop_duplicates(subset=['ID Сайта'])

In [114]:
df['Дата закупки'] = pd.to_datetime(df['Дата закупки'], dayfirst=True, format='mixed')

In [115]:
df['Старая закупка (до 31.12.2022)'] = np.where(
    df['Дата закупки'] <= pd.to_datetime('2022-12-31', dayfirst=True, format='mixed'), 1, 0
)

In [116]:
df['Новая закупка (с 01.01.2023)'] = np.where(
    df['Дата закупки'] >= pd.to_datetime('2023-01-01', dayfirst=True, format='mixed'), 1, 0
)

In [117]:
data = df.groupby(by = 'ID Сайта', as_index=False)[['Старая закупка (до 31.12.2022)', 'Новая закупка (с 01.01.2023)']].sum()

In [118]:
data['Дата закупки'] = np.where(
    (data['Новая закупка (с 01.01.2023)'] > 0) & (data['Старая закупка (до 31.12.2022)'] > 0), 'Старые и новые',
    np.where(
        (data['Новая закупка (с 01.01.2023)'] > 0) & (data['Старая закупка (до 31.12.2022)'] == 0), 'Новые', 
        np.where(
            (data['Новая закупка (с 01.01.2023)'] == 0) & (data['Старая закупка (до 31.12.2022)'] > 0), 'Старые', 'Старые'
        )
    )
)

In [119]:
data = data.drop(columns=['Старая закупка (до 31.12.2022)', 'Новая закупка (с 01.01.2023)'])

In [120]:
result = result.merge(data, how='left', on='ID Сайта')

In [121]:
result = result.drop(columns=['Дата закупки_x'])

In [122]:
result.columns = ['Код склада', 'Код товара', 'Артикул поставщика', 'Внутренний артикул',
    'ID Сайта', 'Товарное направление', 'Товарная группа',
    'Товарный кластер', 'Проба', 'Размер изделия', 'Тип изделия 1',
    'Тип изделия 2', 'Тип изделия 3', 'Дизайн', 'Бренд',
    'Гендерный признак', 'Ценовая корзина', 'Группа цен',
    'Цена Розн., за шт', 'Чистый вес', 'Цена изделия на бирке',
    'Скидка на изделии', 'Запрет скидки',
    'Цена Закупки за 1 шт. (упр сс с НДС за единицу)', 'Опубликован',
    'Ссылка на изображение', 'Остаток', 'Количество продаж 14.03-19.06',
    'Сумма продаж 14.03-19.06', 'Чистый вес макс', 'Чистый вес медиана',
    'Чистый вес мин', 'Чистый вес размах, %', 'Цена изделия на бирке макс',
    'Цена изделия на бирке медиана', 'Цена изделия на бирке мин',
    'Базовая скидка макс', 'Базовая скидка мин', 'Цена со скидками макс',
    'Цена со скидками медиана', 'Цена со скидками мин',
    'Цена со скидками размах, %', 'Себестоимость макс',
    'Себестоимость медиана', 'Себестоимость мин', 'Себестоимость размах, %',
    'Наценка макс', 'Наценка медиана', 'Наценка мин', 'Запрет к размещению',
    'Цена со скидками Ecom', 'Сумма в себестоимости на остатках',
    'Цена за грамм макс', 'Цена за грамм медиана', 'Цена за грамм мин',
    'Количество с запрещенной скидкой', 'Количество с разрешенной скидкой',
    'Дата закупки']

In [123]:
result[['Чистый вес размах, %']]

,"Чистый вес размах, %"
0,0.244370
1,0.928866
2,0.342493
3,0.410453
4,0.261246
...,...
44602,0.000000
44603,0.000000
44604,0.000000
44605,0.000000


In [124]:
result['Размах по весу'] = np.where(
    result['Чистый вес размах, %'] >= 0.3, '3. Большой размах (>30%)',
    np.where(
        result['Чистый вес размах, %'] >= 0.1, '2. Средний размах (10-30%)', '1. Небольшой размах (<10%)'
    )
)

In [125]:
result['Размах по себестоимости'] = np.where(
    result['Себестоимость размах, %'] >= 0.5, '3. Большой размах (>50%)',
    np.where(
        result['Себестоимость размах, %'] >= 0.2, '2. Средний размах (20-50%)', '1. Небольшой размах (<10%)'
    )
)

In [126]:
result['Размах по цене со скидками'] = np.where(
    result['Цена со скидками размах, %'] >= 0.5, '3. Большой размах (>50%)',
    np.where(
        result['Цена со скидками размах, %'] >= 0.2, '2. Средний размах (20-50%)', '1. Небольшой размах (<10%)'
    )
)

In [129]:
result['Цена за грамм размах, %'] = (result['Цена за грамм макс'] / result['Цена за грамм мин'] - 1) 

In [130]:
result['Размах по цене за грамм'] = np.where(
    result['Цена за грамм размах, %'] >= 0.3, '3. Большой размах (>30%)',
    np.where(
        result['Цена за грамм размах, %'] >= 0.1, '2. Средний размах (10-30%)', '1. Небольшой размах (<10%)'
    )
)

In [131]:
result['Скидка разрешена'] = result['Количество с запрещенной скидкой'] + result['Количество с разрешенной скидкой']

In [132]:
result['Скидка разрешена'] = np.where(
    result['Скидка разрешена'].astype(str).str.contains('nan'), 'Да или нет', 'Да и нет'
)

In [133]:
result.columns

Index(['Код склада', 'Код товара', 'Артикул поставщика', 'Внутренний артикул',
       'ID Сайта', 'Товарное направление', 'Товарная группа',
       'Товарный кластер', 'Проба', 'Размер изделия', 'Тип изделия 1',
       'Тип изделия 2', 'Тип изделия 3', 'Дизайн', 'Бренд',
       'Гендерный признак', 'Ценовая корзина', 'Группа цен',
       'Цена Розн., за шт', 'Чистый вес', 'Цена изделия на бирке',
       'Скидка на изделии', 'Запрет скидки',
       'Цена Закупки за 1 шт. (упр сс с НДС за единицу)', 'Опубликован',
       'Ссылка на изображение', 'Остаток', 'Количество продаж 14.03-19.06',
       'Сумма продаж 14.03-19.06', 'Чистый вес макс', 'Чистый вес медиана',
       'Чистый вес мин', 'Чистый вес размах, %', 'Цена изделия на бирке макс',
       'Цена изделия на бирке медиана', 'Цена изделия на бирке мин',
       'Базовая скидка макс', 'Базовая скидка мин', 'Цена со скидками макс',
       'Цена со скидками медиана', 'Цена со скидками мин',
       'Цена со скидками размах, %', 'Себестои

In [134]:
result['Цена изделия на бирке размах, %'] = (result['Цена изделия на бирке макс'] / result['Цена изделия на бирке мин'] - 1) 

In [135]:
result['Размах цена изделия на бирке'] = np.where(
    result['Цена изделия на бирке размах, %'] >= 0.5, '3. Большой размах (>50%)',
    np.where(
        result['Цена изделия на бирке размах, %'] >= 0.2, '2. Средний размах (20-50%)', '1. Небольшой размах (<20%)'
    )
)

In [136]:
result = result.drop(columns = ['Код склада', 'Чистый вес', 'Цена изделия на бирке', 'Скидка на изделии', 'Запрет скидки', 'Цена Закупки за 1 шт. (упр сс с НДС за единицу)',
                                'Запрет скидки'])

In [137]:
result = result[['Код товара', 'Артикул поставщика', 'Внутренний артикул', 'ID Сайта',
    'Товарное направление', 'Товарная группа', 'Товарный кластер', 'Проба',
    'Размер изделия', 'Тип изделия 1', 'Тип изделия 2', 'Тип изделия 3',
    'Дизайн', 'Бренд', 'Гендерный признак', 'Ценовая корзина', 'Группа цен',
    'Опубликован', 'Ссылка на изображение', 'Остаток', 'Цена Розн., за шт',
    'Количество продаж 14.03-19.06', 'Сумма продаж 14.03-19.06',
    'Чистый вес макс', 'Чистый вес медиана', 'Чистый вес мин',
    'Чистый вес размах, %', 'Цена изделия на бирке макс',
    'Цена изделия на бирке медиана', 'Цена изделия на бирке мин', 'Цена изделия на бирке размах, %',
    'Базовая скидка макс', 'Базовая скидка мин', 'Цена со скидками макс',
    'Цена со скидками медиана', 'Цена со скидками мин',
    'Цена со скидками размах, %', 'Себестоимость макс',
    'Себестоимость медиана', 'Себестоимость мин', 'Себестоимость размах, %',
    'Наценка макс', 'Наценка медиана', 'Наценка мин', 'Запрет к размещению',
    'Цена со скидками Ecom', 'Сумма в себестоимости на остатках',
    'Цена за грамм макс', 'Цена за грамм медиана', 'Цена за грамм мин', 'Цена за грамм размах, %',
    'Количество с запрещенной скидкой', 'Количество с разрешенной скидкой',
    'Размах по весу', 'Размах по себестоимости',
    'Размах по цене со скидками', 'Размах по цене за грамм', 'Размах цена изделия на бирке', 'Скидка разрешена', 'Дата закупки']]

In [138]:
tov = result.groupby(by = 'Товарная группа')['Остаток'].sum()

In [139]:
tov

Товарная группа
БК АКСЕССУАРЫ          98.0
БК КОЛЬЦА           68007.0
БК КОЛЬЦА ОБРУЧ    218880.0
БК ПЕЧАТКИ          18568.0
БК ПОДВЕС ДЕКОР     44555.0
БК ПОДВЕС КУЛЬТ    180520.0
БК СЕРЬГИ          217374.0
ДК КАМНИ                6.0
ДК КОЛЬЦА          239842.0
ДК КОЛЬЦА ОБРУЧ     33501.0
ДК ПЕЧАТКИ           6062.0
ДК ПОДВЕСКИ         65128.0
ДК СЕРЬГИ          167163.0
ИФ АКСЕССУАРЫ         433.0
ИФ КОЛЬЦА          242694.0
ИФ КОЛЬЦА ОБРУЧ     27511.0
ИФ ПЕЧАТКИ          26535.0
ИФ ПОДВЕС ДЕКОР    123752.0
ИФ ПОДВЕС КУЛЬТ     38167.0
ИФ СЕРЬГИ          331658.0
ОПТ БК               6164.0
ОПТ ИФ              12540.0
ОПТ КОНГО            1771.0
ОПТ ОБРУЧИ           4027.0
ОПТ ПДК              2938.0
ПДК КОЛЬЦА         186787.0
ПДК ПЕЧАТКИ         11563.0
ПДК ПОДВЕСКИ       102319.0
ПДК СЕРЬГИ         232844.0
ЦБ БРАСЛЕТЫ        228330.0
ЦБ ЦЕПИ            348324.0
Name: Остаток, dtype: float64

In [140]:
tov_sales = result.groupby(by = 'Товарная группа')['Количество продаж 14.03-19.06'].sum()

In [141]:
result['Доля % остатка ID отосительно остатка ТГ'] = np.nan
for i in result.index: 
    t = result['Товарная группа'][i]
    result['Доля % остатка ID отосительно остатка ТГ'] = result['Остаток'] / tov[t] 

In [142]:
result.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 44607 entries, 0 to 44606
Data columns (total 61 columns):
 #   Column                                    Non-Null Count  Dtype  
---  ------                                    --------------  -----  
 0   Код товара                                44607 non-null  object 
 1   Артикул поставщика                        44607 non-null  object 
 2   Внутренний артикул                        44607 non-null  object 
 3   ID Сайта                                  44607 non-null  object 
 4   Товарное направление                      44607 non-null  object 
 5   Товарная группа                           44607 non-null  object 
 6   Товарный кластер                          44607 non-null  object 
 7   Проба                                     44607 non-null  object 
 8   Размер изделия                            44607 non-null  object 
 9   Тип изделия 1                             44607 non-null  object 
 10  Тип изделия 2                     

In [144]:
result[['Цена изделия на бирке макс', 'Цена изделия на бирке мин', 'Цена изделия на бирке медиана']]

,Цена изделия на бирке макс,Цена изделия на бирке мин,Цена изделия на бирке медиана
0,93658.0,62896.0,62896.0
1,130109.0,48404.0,87062.0
2,169978.0,72238.0,151742.0
3,104835.0,56497.0,82717.0
4,47137.0,30916.0,42223.0
...,...,...,...
44602,14695.0,14695.0,14695.0
44603,14490.0,14490.0,14490.0
44604,11513.0,11513.0,11513.0
44605,10450.0,10450.0,10450.0


In [145]:
result.columns

Index(['Код товара', 'Артикул поставщика', 'Внутренний артикул', 'ID Сайта',
       'Товарное направление', 'Товарная группа', 'Товарный кластер', 'Проба',
       'Размер изделия', 'Тип изделия 1', 'Тип изделия 2', 'Тип изделия 3',
       'Дизайн', 'Бренд', 'Гендерный признак', 'Ценовая корзина', 'Группа цен',
       'Опубликован', 'Ссылка на изображение', 'Остаток', 'Цена Розн., за шт',
       'Количество продаж 14.03-19.06', 'Сумма продаж 14.03-19.06',
       'Чистый вес макс', 'Чистый вес медиана', 'Чистый вес мин',
       'Чистый вес размах, %', 'Цена изделия на бирке макс',
       'Цена изделия на бирке медиана', 'Цена изделия на бирке мин',
       'Цена изделия на бирке размах, %', 'Базовая скидка макс',
       'Базовая скидка мин', 'Цена со скидками макс',
       'Цена со скидками медиана', 'Цена со скидками мин',
       'Цена со скидками размах, %', 'Себестоимость макс',
       'Себестоимость медиана', 'Себестоимость мин', 'Себестоимость размах, %',
       'Наценка макс', 'Наце

In [146]:
result['Доля % относительно продаж ТГ 14.03-19.06'] = np.nan
for i in result.index: 
    t = result['Товарная группа'][i]
    result['Доля % относительно продаж ТГ 14.03-19.06'] = result['Количество продаж 14.03-19.06'] / tov_sales[t] 

In [147]:

result.to_excel(r'C:\Users\Trenkin.Sergey\Desktop\notebooks\result.xlsx', index=False)

In [148]:
result.to_excel(r'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\Исследование цен\\в разрезе ID исправленный.xlsx', index=False)

In [149]:
result['Товарное направление'].value_counts()

Товарное направление
ПДК    15220
ДК     13820
ИФ      8328
БК      4796
ЦБ      2443
Name: count, dtype: int64

In [150]:
for i in result['Товарное направление'].unique():
    result[result['Товарное направление'] == i].to_excel(rf'\\gold585.int\uk\Общее хранилище файлов\Служба аналитики\Тренкин Сергей\Исследование цен\\Исследование цен для {i} в разрезе ID.xlsx', index=False) 